### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯
## Laboratory 1 Solution
### Cooperating with: Luca Lodesani (s346978)
### ⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯⎯

In [ ]:
import numpy as np

In [67]:
# Problem 1 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))


In [79]:
# Problem 2 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [26]:
# Problem 3 setup
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS,  size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

## Base Hill Climbing 

In [ ]:
# --- Hill climbing setup ---
solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)

# fitness
def value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity
def is_valid(sol):
    # max 1 bag for each item
    if np.any(sol.sum(axis=0) > 1):
        return False

    # check weight constraints for each knapsack
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

def tweak(sol):
    new_solution = solution.copy()
    # randomly select knapsack and item
    k = rng.integers(0, NUM_KNAPSACKS)
    i = rng.integers(0, NUM_ITEMS)
    #tweak: flip random item in random knapsack
    new_solution[k, i] = not new_solution[k, i]
    return new_solution

# --- Hill climbing loop ---
iterations = 100000
for _ in range(iterations):

    new_solution = tweak(solution)

    # generate new solution if invalid
    if not is_valid(new_solution):
        continue
    
    # update solution if better or equal
    if value(new_solution) >= value(solution):
        solution = new_solution

# --- Output results ---
total_value = value(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Zaino {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Zaino {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")


Total value: 35332
Zaino 1 value: 3056
Zaino 2 value: 3042
Zaino 3 value: 2202
Zaino 4 value: 3878
Zaino 5 value: 3650
Zaino 6 value: 3827
Zaino 7 value: 3292
Zaino 8 value: 2638
Zaino 9 value: 4707
Zaino 10 value: 5040
Zaino 1 weights: [4514 3200 3053 3140 2346 3147 2017 2737 1922 1929], constraints: [5837 5754 9351 3205 4450 3447 3952 9256 9525 2357]
Zaino 2 weights: [3593 2424 3539 2589 2686 3178 3067 2646 2850 4058], constraints: [7056 3862 3540 4336 3555 5921 9964 6691 7785 5946]
Zaino 3 weights: [2067 2637 3569 2362 3916 2237 2584 2730 3094 2361], constraints: [2111 2672 4150 3949 8724 8748 8832 7100 6449 7193]
Zaino 4 weights: [1941 1335 3239 3049 2786 2435 2490 2090 2442 2962], constraints: [5254 7361 8355 8103 6910 2464 7799 4932 7949 6316]
Zaino 5 weights: [2991 4606 2186 4698 4627 4508 5336 3032 3461 2363], constraints: [9049 4707 9451 8755 4648 5860 6250 8149 7513 8816]
Zaino 6 weights: [2459 4028 4379 3115 3351 3058 2947 3138 2236 3148], constraints: [4958 6038 5446 9276 4

# Tabu Search Hill Climbing

In [ ]:
NUM_SAMPLES = 10  # number of neighbors to sample

In [ ]:
# --- Hill climbing setup ---
solution = np.zeros((NUM_KNAPSACKS, NUM_ITEMS), dtype=bool)

# fitness
def value(sol):
    return VALUES[np.any(sol, axis=0)].sum()

# validity
def is_valid(sol):
    # max 1 bag for each item
    if np.any(sol.sum(axis=0) > 1):
        return False

    # check weight constraints for each knapsack
    return np.all([WEIGHTS[sol[k]].sum(axis=0) <= CONSTRAINTS[k] for k in range(NUM_KNAPSACKS)])

def tweak(sol):
    new_solution = sol.copy()
    # randomly select knapsack and item
    k = rng.integers(0, NUM_KNAPSACKS)
    i = rng.integers(0, NUM_ITEMS)
    #tweak: flip random item in random knapsack
    new_solution[k, i] = not new_solution[k, i] 
    return new_solution

# --- Hill climbing loop ---
iterations = 100000
tabu_list = set()
for _ in range(iterations):

    possible_solutions = [tweak(solution) for _ in range(NUM_SAMPLES)]
    candidates = [sol for sol in possible_solutions if is_valid(sol) and sol.flatten().tobytes() not in tabu_list]

    # if no valid candidates, continue
    if not len(candidates):
        continue

    new_solution = max(candidates, key=value)
    
    # update solution if better or equal
    if value(new_solution) >= value(solution):
        tabu_list.add(solution.flatten().tobytes())
        solution = new_solution

# --- Output results ---
total_value = value(solution)
values_per_knapsack = [VALUES[solution[k]].sum() for k in range(NUM_KNAPSACKS)]

print("Total value:", total_value)
for k, v in enumerate(values_per_knapsack):
    print(f"Zaino {k+1} value: {v}")

for k in range(NUM_KNAPSACKS):
    total_weights = WEIGHTS[solution[k]].sum(axis=0)
    print(f"Zaino {k+1} weights: {total_weights}, constraints: {CONSTRAINTS[k]}")
    


Total value: 40077
Zaino 1 value: 3055
Zaino 2 value: 4160
Zaino 3 value: 4238
Zaino 4 value: 4347
Zaino 5 value: 5797
Zaino 6 value: 4026
Zaino 7 value: 2784
Zaino 8 value: 4052
Zaino 9 value: 3339
Zaino 10 value: 4279
Zaino 1 weights: [4027 3854 3149 3082 3599 3217 3342 3252 3020 2266], constraints: [5837 5754 9351 3205 4450 3447 3952 9256 9525 2357]
Zaino 2 weights: [4108 3768 3262 3546 2795 2808 1865 3133 2921 3421], constraints: [7056 3862 3540 4336 3555 5921 9964 6691 7785 5946]
Zaino 3 weights: [1699 2671 2920 3566 3400 2530 2474  764 1947 2913], constraints: [2111 2672 4150 3949 8724 8748 8832 7100 6449 7193]
Zaino 4 weights: [1863 3666 2625 2699 2774 2425 3364 3235 3923 1272], constraints: [5254 7361 8355 8103 6910 2464 7799 4932 7949 6316]
Zaino 5 weights: [4627 4686 4415 5354 4546 4706 6247 3980 4082 4888], constraints: [9049 4707 9451 8755 4648 5860 6250 8149 7513 8816]
Zaino 6 weights: [2709 2474 3743 3919 4412 4000 2805 2483 4200 4097], constraints: [4958 6038 5446 9276 4